In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')


def linear_fit(x, a, b):
    
    return a + b * x



# load data

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
os.listdir(os.path.join(RESULTPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)
#test_metadata = pd.DataFrame(test_data.metadata)

# pick signal, frequency modulate to a lower energy (higher frequency)

In [ ]:
signal_angle = 85.
signal_energy = 18575
var = 1.38e-23 * 10 * 50 * 200e6

hz_per_eV = 49.1e3
nsample = 3 * 8192
hz_per_bin = 200e6 / nsample

eV_shift = 0
f_shift = eV_shift * hz_per_eV
bin_shift = int(f_shift // hz_per_bin)

#sns.set_theme(style='whitegrid', context='talk')
#fig = plt.figure(figsize=(13, 8))
#ax = fig.add_subplot(1,1,1)

signal_index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]
signal = template_data.data[signal_index, :]
signal = signal.reshape((60, 8192 * 3))

#ax.plot(np.fft.fft(signal[0, :]))
signal = np.roll(np.fft.fftshift(np.fft.fft(signal, axis=-1), axes=-1), bin_shift, axis=-1)
#ax.plot(signal[0, :])
signal = np.fft.ifft(np.fft.ifftshift(signal, axes=-1), axis=-1).flatten()
#ax.plot(abs(signal[0:1024]))

# normalize signal, calculate scores

In [ ]:
norm = 1 / np.sqrt(var * np.vdot(signal, signal))
template = signal * norm

signal_shape = template_data.data.shape
nsignal = signal_shape[0]
indxs = np.arange(0, nsignal, 1)

split_indxs = np.array_split(indxs, 3)
scores = np.zeros(nsignal)
for split in split_indxs:
    print(split)

    signals = template_data.data[split, :]
    scores[split] = abs(np.matmul(signals, template.reshape((template.size, 1)).conjugate()).squeeze())

#templates = (signals.T * norm).T

#print(scores.shape)

# find sorting indicies with metadata

In [ ]:
energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

#test_energies = np.array(test_metadata['energy'].array).squeeze()
#test_angles = np.array(test_metadata['theta_min'].array).squeeze()

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorting_indexes = []

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    sorting_indexes.append(template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0])
    
sorting_indexes = np.array(sorting_indexes)

print(sorting_indexes.shape)

In [ ]:
print(unique_energy, unique_angle)

# plot 

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

#signal_angle = 89.6
#signal_energy = 18595

#signal_index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]
#print(signal_index)

#signal_index = 401

#signal_energy = test_metadata['energy'].iloc[signal_index]
#signal_angle = test_metadata['theta_min'].iloc[signal_index]

#print(signal_energy)
#print(signal_angle)

plot_scores = abs(scores[sorting_indexes].reshape(energy_grid.shape))

peak_coords_1 = np.flip(np.argwhere(plot_scores >= 6))
print(unique_energy[peak_coords_1[:, 0]])
print(unique_angle[peak_coords_1[:, 1]])
img = ax.imshow(plot_scores, aspect='auto', cmap=cmap, extent = (18575, 18600, 90, 84))
fig.colorbar(img)

test_signal_energy = 18584
test_signal_angle = 89.85

#ax.plot(signal_energy, signal_angle, 'r*', markersize=8, label='Signal Parameters')
#ax.plot(test_signal_energy, test_signal_angle, 'r*', markersize=8, label='Signal Parameters')

#ax.legend(loc=1)
ax.set_xlabel('Template Energy (eV)')
ax.set_ylabel('Template Pitch Angle (deg)')

name = '211007_84_100_template_on_template_example_10'

plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, name))

# plot 2

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

signal_angle = 89.6
signal_energy = 18596

signal_index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]
print(signal_index)

#signal_index = 401

#signal_energy = test_metadata['energy'].iloc[signal_index]
#signal_angle = test_metadata['theta_min'].iloc[signal_index]

#print(signal_energy)
#print(signal_angle)

plot_scores = scores[signal_index, sorting_indexes].reshape(energy_grid.shape)

peak_coords_2 = np.flip(np.argwhere(plot_scores >= 6))

img = ax.imshow(plot_scores, aspect='auto', cmap=cmap, extent = (18590, 18600, 90, 89))
fig.colorbar(img)

ax.plot(signal_energy, signal_angle, 'r*', markersize=8, label='Signal Parameters')
ax.plot(18595.25, 89.6125, 'r*', markersize=8, label='Signal Parameters')
#ax.legend(loc=1)

ax.set_xlabel('Template Energy (eV)')
ax.set_ylabel('Template Pitch Angle (deg)')

name = '211007_84_100_template_on_template_example_10'

plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, name))

# plot peaks

In [ ]:
sns.set_theme(context='paper', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(peak_coords_1[:, 0], peak_coords_1[:, 1], '.')
ax.plot(peak_coords_2[:, 0], peak_coords_2[:, 1], '.')